# Notebook E-tivity 1 CE4021

Student name: Bartlomiej Mlynarkiewicz

Student ID: 17241782

<hr style="border:2px solid gray"> </hr>

## Imports

If you believe required imports are missing, please contact your moderator.

<hr style="border:2px solid gray"> </hr>

## Task

You may add as many cells as you require to complete this task. Refer to the Rubric for E-tivity 1 to see how you will be assessed.

# Option 1: Symbolic calculation of the derivative

Write 2 Python functions:

1. *The first one to symbolically calculate the derivative of a polynomial with one variable. The input should be a* *polynomial and the output should also be a polynomial.*

2. *The second one to evaluate (i.e. get the numerical value) of a polynomial for a given value of its variable. The* *input should be a polynomial and a value (point at which to evaluate the polynomial). The output should be a scalar.*

*Test your code with a few salient polynomials (minimum of 3) for which you have calculated the derivative of these* *polynomials manually.*

### Symbolic calculation of the derivative

To perform a symbolic differentiation of a single varaible, the basic rule will be used:

$$
\frac{d}{dx} ax^n = n\times ax^{n-1}
$$

A polynomial is a list of terms which must satisfy the basic rules to be a polynomial.

Polynomials can be represnted as a list in reverse exponential order such that the list values are the coefficients and the index positions are the powers.

In this way the following polynomial can be written as:

$$
4x3+3x2-2x+4 \to [4,2,3,4]
$$

### Assumptions
1. All equations are fully expanded and simplified.
2. All exponents are positive integers.

I've decided to store the polynomial as a list of coefficeints rather than storing coefficients as a list of tuples or a map by having the power as the key and the coefficeints as its values as suggested in the etivity forum. 

I think using a int list is just as a simple as all the afforementioned approaches as they all still require to be itterated over to be evaluated to a single value.

In [506]:
def derive_polynomial(polynomial:list[int]) -> list[int]:
    """Returns a list of coefficients of the derived polynomail.

        Parameters
        ----------
        polynomial : list[int]
            A list of coefficients of a polynomial.
            
        Returns
        -------
        list
            Returns a list of coefficients of the derived polynomail.
    """ 
    derived_cofficients:list[int] = [0] * len(polynomial)
    for exponent, cofficient in enumerate(polynomial):
        derived_exponent = exponent-1
        derived_cofficient = cofficient*exponent
        if exponent != 0 and cofficient != 0: 
            derived_cofficients[derived_exponent] = derived_cofficient
    return derived_cofficients

In [507]:
def evalute_polynomial(polynomial:list[int], val_of_x:int) -> int:
    """Evaluates a polynomail using a list of its coefficients.

        Parameters
        ----------
        polynomial : list[int]
            A list of coefficients of a polynomial.
            
        val_of_x : int
            Value which is substituted for x to evaluate a polynomial to result in a scalar value.
            
        Returns
        -------
        int
            Resulting scalar after evaluating the polynomial expression.
    """ 
    scalar:int = 0
    for exponent, cofficient in enumerate(polynomial):
        scalar += cofficient * pow(val_of_x, exponent)
    return scalar

## Differentiate polynomail using a list of coefficients

In [508]:
def derive_polynomial_using_list(polynomial:list[int], val_of_x:int, derrivative:int = 1) -> list:
    """Returns a prettified derived polynomail and its evaluated value

        Parameters
        ----------
        polynomial : list[int]
            A polynomial in the form of list of coefficients
            
        val_of_x : int
            The value for which the derived polynomail is to be evaluated for
        
        derrivative : int, optional
            The derrivative number of the polynomail
            
        Returns
        -------
        list
            Returns a pretified polynomail with its evaluated value along with a derived list of coefficients

        Raises
        ------
        InvalidPolynomial
            If the polynomail is empty
    """
    
    if not len(polynomial):
        raise InvalidPolynomial("Invalid polynomial has been passed in.")
        
    derived_cofficients:list[int] = [0] * len(polynomial)
    derived_polynomial:list[str] = []
        
    derived_polynomial:list[int] = derive_polynomial(polynomial)
    evaluated_polynomial:int = evalute_polynomial(derived_polynomial, val_of_x)
            
    prettified_polynomail = prettify_polynomail(derived_polynomial, derrivative, reverse_polynomial=True)
    inital_prettified_polynomail = prettify_polynomail(polynomial, reverse_polynomial=True)

    return f"{derrivative}. {inital_prettified_polynomail[0]} -> f{prettified_polynomail[1]}(x) = {prettified_polynomail[0]} -> f{prettified_polynomail[1]}({val_of_x}) = {evaluated_polynomial}", derived_polynomial

The `find_highest_power` is a helper function of `derive_polynomial_using_string` used to find the largest exponent in the polynomial to create a list of the size of the largest exponent to store the coefficients of the polynomail

In [509]:
def find_highest_power(polynomial:str) -> int:
    """Returns the greatest expoent parsed from the polynomial

        Parameters
        ----------
        polynomial : str
            A polynomial in its string representation.
            
        Returns
        -------
        int
            Returns the greatest expoent parsed from the polynomial
    """
    power = 0
    for term in polynomial:
        term = term.split('x')
        exponent = 0 if len(term) == 1 else 1 if term[1] == '' else int(term[1])
        if exponent > power:
            power = exponent 
    return power

I've decided to add this function to parse the polynomial in its string format. From a UX perspective, allowing the user to pass in the polynomial in its string format is least confusing. 

For the purposes of this task, I'm not implementing any string validation and assuming the polynomial is passed in its expected form.

## Differentiate polynomail using its string format

In [510]:
def derive_polynomial_using_string(polynomial:str, val_of_x:int, derrivative:int = 1) -> list:
    """Returns a prettified derived polynomail and its evaluated value

        Parameters
        ----------
        polynomial : list[int]
            A polynomial in the form of list of coefficients
            
        val_of_x : int
            The value for which the derived polynomail is to be evaluated for
        
        derrivative : int, optional
            The derrivative number of the polynomail
            
        Returns
        -------
        list
            Returns a pretified polynomail with its evaluated value along with a derived list of coefficients

        Raises
        ------
        InvalidPolynomial
            If the polynomail is empty
    """
    
    if not len(polynomial):
        raise InvalidPolynomial("Invalid polynomial has been passed in.")
        
    polynomial:list[str] = polynomial.replace("^","").replace(" - ", "+-").replace(" ","").split("+")
    
    power = find_highest_power(polynomial)
    polynomial_list:list[int] = [0] * (power + 1)

    derived_polynomial:list[int] = []
    scalar:int = 0

    for term in polynomial:
        term = term.split('x')
        cofficient = 1 if term[0] == '' else int(term[0])
        exponent = 0 if len(term) == 1 else 1 if term[1] == '' else int(term[1])
        polynomial_list[exponent] = cofficient
        
    derived_polynomial:list[int] = derive_polynomial(polynomial_list)
    evaluated_polynomial:int = evalute_polynomial(derived_polynomial, val_of_x)
                    
    prettified_polynomail = prettify_polynomail(derived_polynomial, derrivative, reverse_polynomial=True)
    inital_prettified_polynomail = prettify_polynomail(polynomial_list, reverse_polynomial=True)
    
    return f"{derrivative}. {inital_prettified_polynomail[0]} -> f{prettified_polynomail[1]}(x) = {prettified_polynomail[0]} -> f{prettified_polynomail[1]}({val_of_x}) = {evaluated_polynomial}", derived_polynomial

# Prettify Polynomail

A prettify function to represent to derived polynomail in its nth derivative.

It reverses the polynomail if polynomail was passed in as a list of coefficients.

In [511]:
def prettify_polynomail(polynomial:list, derivative:int = 1, reverse_polynomial:bool = False) -> list:
    """Returns a pretified derived polynomail

        Parameters
        ----------
        polynomial : list[str]
            The derived polynomail is passed in.
            
        derivative_no : int, optional
            The value for which the derived polynomail is to be evaluated for
        
        reverse_polynomial : bool, optional
            A boolean flag if to reverse the polynomail, is reversed when polynomial was constructed from a list of its coefficients
            
        Returns
        -------
        list
            Returns a pretified polynomail with its evaluated value along with a derived list of coefficients
    """      
    pretified_polynomial:list[str] = []
    derivative_no:str = derivative * "'"
            
    for exponent, cofficient in enumerate(polynomial):
        if cofficient != 0:
            cofficient = str(cofficient)
            pretified_polynomial.append((cofficient + (f"x^{exponent}" if (exponent) > 1 else ("x" if (exponent) == 1 else ""))).replace("-1x","-x").replace("1x","x"))
            
    if reverse_polynomial:
        pretified_polynomial.reverse()
        
    prettified_polynomial = " + ".join(pretified_polynomial).replace(" + -"," - ")
    prettified_polynomial = 0 if not len(prettified_polynomial) else prettified_polynomial
    
    return prettified_polynomial, derivative_no

# Tests

Verify that prettify_polynomail returns the polynomail in the required as expected

In [512]:
print(f"1. {prettify_polynomail([0, -1, 0, 2, -4, 1])[0]}")
print(f"2. {prettify_polynomail([0, -1])[0]}")
print(f"3. {prettify_polynomail([0])[0]}")

1. -x + 2x^3 - 4x^4 + x^5
2. -x
3. 0


In [513]:
print(f"1. [0, -1, 0, 2, -4, 1] -> {derive_polynomial([0, -1, 0, 2, -4, 1])}")
print(f"2. [0, -1] -> {derive_polynomial([0, -1])}")
print(f"3. [0] -> {derive_polynomial([0])}")

1. [0, -1, 0, 2, -4, 1] -> [-1, 0, 6, -16, 5, 0]
2. [0, -1] -> [-1, 0]
3. [0] -> [0]


## Tests of derive_polynomial_using_list

In [514]:
derived_polynomial = derive_polynomial_using_list([0,0,0,-44,100,1,1,2], 2)
print(derived_polynomial[0])

1. 2x^7 + x^6 + x^5 + 100x^4 - 44x^3 -> f'(x) = 14x^6 + 6x^5 + 5x^4 + 400x^3 - 132x^2 -> f'(2) = 3840


Import `random` to generate a random array in the range -100 - 100 only for test purposes.

In [515]:
import random 

random_int_array = []
for i in range(100):
    random_int_array.append(random.randint(-100, 100))
    
print(f"Generate array: {random_int_array}")

Generate array: [1, -21, 40, -64, -40, 11, -53, -33, 63, 11, 35, 93, 39, -32, -86, 8, 48, 67, 22, -5, 84, 71, 38, 33, 96, -86, 17, 98, 54, 42, 60, 61, -73, -21, -70, -97, -54, 90, 88, 58, -1, -80, 32, 7, -31, -21, 69, 33, -50, 17, 40, 73, -44, -73, -82, 46, -35, -31, 75, 82, 9, -100, -72, 37, -46, 8, 8, -46, 19, -92, -56, -53, -94, -60, 30, 100, -64, -25, -10, 63, 14, -51, -47, -15, 90, -22, 80, 4, 86, 42, -52, -16, 60, -6, -13, 38, -61, -67, -76, 35]


In [516]:
derived_polynomial = derive_polynomial_using_list(random_int_array, 2)
print(derived_polynomial[0])

1. 35x^99 - 76x^98 - 67x^97 - 6x^96 + 38x^95 - 13x^94 - 6x^93 + 60x^92 - 16x^91 - 52x^90 + 42x^89 + 86x^88 + 4x^87 + 80x^86 - 22x^85 + 90x^84 - 15x^83 - 47x^82 - 5x^81 + 14x^80 + 63x^79 - 10x^78 - 25x^77 - 64x^76 + 100x^75 + 30x^74 - 60x^73 - 94x^72 - 53x^71 - 56x^70 - 92x^69 + 19x^68 - 46x^67 + 8x^66 + 8x^65 - 46x^64 + 37x^63 - 72x^62 - 100x^61 + 9x^60 + 82x^59 + 75x^58 - 3x^57 - 35x^56 + 46x^55 - 82x^54 - 73x^53 - 44x^52 + 73x^51 + 40x^50 + 17x^49 - 50x^48 + 33x^47 + 69x^46 - 2x^45 - 3x^44 + 7x^43 + 32x^42 - 80x^41 - x^40 + 58x^39 + 88x^38 + 90x^37 - 54x^36 - 97x^35 - 70x^34 - 2x^33 - 73x^32 + 6x^31 + 60x^30 + 42x^29 + 54x^28 + 98x^27 + 17x^26 - 86x^25 + 96x^24 + 33x^23 + 38x^22 + 7x^21 + 84x^20 - 5x^19 + 22x^18 + 67x^17 + 48x^16 + 8x^15 - 86x^14 - 32x^13 + 39x^12 + 93x^11 + 35x^10 + 1x^9 + 63x^8 - 33x^7 - 53x^6 + 1x^5 - 40x^4 - 64x^3 + 40x^2 - 2x + 1 -> f'(x) = 3465x^98 - 7448x^97 - 6499x^96 - 5856x^95 + 3610x^94 - 1222x^93 - 558x^92 + 5520x^91 - 1456x^90 - 4680x^89 + 3738x^88 + 756

In [517]:
derived_polynomial = derive_polynomial_using_list([1, 2, 0, -3,0,-45,0,0,0,1], 2)
print(derived_polynomial[0])
for i in range(1,10):
    derived_polynomial = derive_polynomial_using_list(derived_polynomial[1], 2, i+1)
    print(derived_polynomial[0])

1. x^9 - 45x^5 - 3x^3 + 2x + 1 -> f'(x) = 9x^8 - 225x^4 - 9x^2 + 2 -> f'(2) = -1330
2. 9x^8 - 225x^4 - 9x^2 + 2 -> f''(x) = 72x^7 - 900x^3 - 18x -> f''(2) = 1980
3. 72x^7 - 900x^3 - 18x -> f'''(x) = 504x^6 - 2700x^2 - 18 -> f'''(2) = 21438
4. 504x^6 - 2700x^2 - 18 -> f''''(x) = 3024x^5 - 5400x -> f''''(2) = 85968
5. 3024x^5 - 5400x -> f'''''(x) = 15120x^4 - 5400 -> f'''''(2) = 236520
6. 15120x^4 - 5400 -> f''''''(x) = 60480x^3 -> f''''''(2) = 483840
7. 60480x^3 -> f'''''''(x) = 181440x^2 -> f'''''''(2) = 725760
8. 181440x^2 -> f''''''''(x) = 362880x -> f''''''''(2) = 725760
9. 362880x -> f'''''''''(x) = 362880 -> f'''''''''(2) = 362880
10. 362880 -> f''''''''''(x) = 0 -> f''''''''''(2) = 0


## Tests of derive_polynomial_using_string

In [518]:
derived_polynomial = derive_polynomial_using_string("", 2)
print(derived_polynomial[0])

NameError: name 'InvalidPolynomial' is not defined

In [519]:
derived_polynomial = derive_polynomial_using_string("3x^3 - 2x^2 + x - 3x^16 + 1", 2)
print(derived_polynomial[0])

1. -3x^16 + 3x^3 - 2x^2 + x + 1 -> f'(x) = -48x^15 + 9x^2 - 4x + 1 -> f'(2) = -1572835


In [520]:
derived_polynomial = derive_polynomial_using_string("3x + 1", 5)
print(derived_polynomial[0])

1. 3x + 1 -> f'(x) = 3 -> f'(5) = 3


In [521]:
derived_polynomial = derive_polynomial_using_string("3", 0)
print(derived_polynomial[0])

1. 3 -> f'(x) = 0 -> f'(0) = 0


In [522]:
derived_polynomial = derive_polynomial_using_string("2x^1000", 2)
print(derived_polynomial[0])

1. 2x^1000 -> f'(x) = 2000x^999 -> f'(2) = 10715086071862673209484250490600018105614048117055336074437503883703510511249361224931983788156958581275946729175531468251871452856923140435984577574698574803934567774824230985421074605062371141877954182153046474983581941267398767559165543946077062914571196477686542167660429831652624386837205668069376000


In [523]:
derived_polynomial = derive_polynomial_using_string("9x^8 - 225x^4 - 9x^2 + 2", 2)
print(derived_polynomial[0])
for i in range(1,10):
    derived_polynomial = derive_polynomial_using_list(derived_polynomial[1], 2, i+1)
    print(derived_polynomial[0])

1. 9x^8 - 225x^4 - 9x^2 + 2 -> f'(x) = 72x^7 - 900x^3 - 18x -> f'(2) = 1980
2. 72x^7 - 900x^3 - 18x -> f''(x) = 504x^6 - 2700x^2 - 18 -> f''(2) = 21438
3. 504x^6 - 2700x^2 - 18 -> f'''(x) = 3024x^5 - 5400x -> f'''(2) = 85968
4. 3024x^5 - 5400x -> f''''(x) = 15120x^4 - 5400 -> f''''(2) = 236520
5. 15120x^4 - 5400 -> f'''''(x) = 60480x^3 -> f'''''(2) = 483840
6. 60480x^3 -> f''''''(x) = 181440x^2 -> f''''''(2) = 725760
7. 181440x^2 -> f'''''''(x) = 362880x -> f'''''''(2) = 725760
8. 362880x -> f''''''''(x) = 362880 -> f''''''''(2) = 362880
9. 362880 -> f'''''''''(x) = 0 -> f'''''''''(2) = 0
10. 0 -> f''''''''''(x) = 0 -> f''''''''''(2) = 0


<hr style="border:2px solid gray"> </hr>

## Reflection

Write your reflection in below cell. With reference to the Rubric for E-tivity 1:
- Provide an accurate description of your code with advantages and disadvantages of design choices.
- Compare your approach to alternative (peer) approaches.
- Clearly describe how you have used your peers' work/input and how this has affected your own understanding / insights.

If you have not used peer input, you may state this, but your submission history in Gitlab should clearly show this is the case.